<a href="https://colab.research.google.com/github/NikolasGialitsis/LDA2vec/blob/master/lda2vec_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!unzip /content/lda2vec.zip
#!ls
!unzip '/content/covid_embeddings.zip'

Archive:  /content/covid_embeddings.zip
   creating: covid_embeddings/
  inflating: covid_embeddings/covid_19_w2v_embeds_30.model  
  inflating: covid_embeddings/covid_19_w2v_embeds_30.model.trainables.syn1neg.npy  
  inflating: covid_embeddings/covid_19_w2v_embeds_30.model.wv.vectors.npy  


## Setup and Preprocessing

Install Dependencies



In [0]:
#!git clone https://github.com/maxent-ai/lda2vec

!pip install spacy
!pip install jellyfish
!pip install -r /content/lda2vec/requirements.txt 
!pip install pylda2vec


In [0]:
!python -m spacy download en

Import Libraries

In [0]:
%cd /content/lda2vec/
import os
import os.path
import pickle
import time
import shelve

import chainer
from chainer import cuda
from chainer import serializers
import chainer.optimizers as O
import numpy as np

from lda2vec import utils
from lda2vec import prepare_topics, print_top_words_per_topic, topic_coherence
from lda2vec import LDA2Vec
from lda2vec import preprocess, Corpus
#changed Preprocess.py line 'nlp = spacy.load("en_core_web_sm")' to solve error en not found
#and removed the import ... as en line 

import logging
logging.basicConfig()
import pickle
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from lda2vec import preprocess, Corpus

/content/lda2vec


**CUDA** usage\
 *change Notebook Settings 'Hardware Acceleration' to 'GPU' to run*

In [0]:
gpu_id = int(os.getenv('CUDA_GPU', 0))
cuda.get_device(gpu_id).use()
print("Using GPU:" + str(gpu_id))

Using GPU:0


**Fetch Data**\
Set Stopwords\
>>**DONE**: replace texts with covid19 collection

In [0]:
import json
with open('/content/db_entries.json') as f:
  data = json.loads(f.read())
titles = []
abstracts = []
concats = []
for entry in data:
  concat = (entry['joint_text']).split('------------------------------')  
  title = concat[0]
  abstract = concat[1]
  #print('title:',title)
  #print('abstract',abstract[:100])
  
  concats.append(concat[0] + ' ' + concat[1])
  titles.append(title)
  abstracts.append(abstract)

Pre-processing

*   Lowercase
*   Remove stopwords from nltk
*   Remove punctuation



In [0]:
import nltk 
import string
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

stop_words = set(stopwords.words('english')) 
texts = []
for text in concats.copy():
  word_tokens = word_tokenize(text) 
  #remove stopwords
  text = ' '.join([w.lower() for w in word_tokens if not w.lower() in stop_words and len(w) >= 3]) 
  #remove punctuation
  #bioclean    = lambda t: re.sub('[.,?;*!%^&_+():-\[\]{}]', '', t.replace('"', '').replace('/', '').replace('\\', '').replace("'", '').strip().lower()).split()

  text = ''.join([w for w in text if w not in string.punctuation])
  print(text[:100])
  texts.append(text)
    


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2019 novel coronavirus infection gastrointestinal tract since end december 2019 cluster patients pne
several suggestion operation colorectal cancer outbreak corona virus disease china pneumonia caused 
potential presymptomatic transmission sarscov2 zhejiang province china 2020 report 2family cluster p
straining system novel coronavirus covid19 preparedness concomitant disasters weeks first confirmed 
risk transportation 2019 novel coronavirus disease wuhan cities china january 2020 china quarantined
advances research cytokine storm mechanism induced corona virus disease 2019 corresponding immunothe
case 2019 novel coronavirus pregnant woman preterm delivery presented case 30week pregnant woman cov
diagnostic therapeutic strategies lung cancer patients outbreak

Preprocessing


*   set max word num per document
*   remove stopwords (aka. words in 'bad' list above)



In [0]:
# Preprocess data
max_length = 10000   # Limit of 10k words per document

Calculate tokens (list) and vocabulary (dictionary)\
OR\
Load them if already calculated

In [0]:
'''if a model is already trained,
skip to the cell loading vocab,corpus,flattened and doc_ids'''
tokens, vocab = preprocess.tokenize(texts, max_length, merge=False,n_threads=4)
'''LOAD TOPICS IF ALREADY COMPUTED'''
#tokens = np.load("tokens.npy")
#vocab = np.load("vocab.npy")

'LOAD TOPICS IF ALREADY COMPUTED'

run next line only if loaded

In [0]:
#vocab = vocab.tolist()

Bag-of-Words and Normalization
* remove really rare words
* downsample frequent words
* remove word gaps for words not in the vocabulary 
* flatten

In [0]:
corpus = Corpus()
# Make a ranked list of rare vs frequent words
corpus.update_word_count(tokens)
corpus.finalize()
# The tokenization uses spaCy indices, and so may have gaps
# between indices for words that aren't present in our dataset.
# This builds a new compact index
compact = corpus.to_compact(tokens)
# Remove extremely rare words
pruned = corpus.filter_count(compact, min_count=2)
# Convert the compactified arrays into bag of words arrays
bow = corpus.compact_to_bow(pruned)
# Words tend to have power law frequency, so selectively
# downsample the most prevalent words
clean = corpus.subsample_frequent(pruned)
# Now flatten a 2D array of document per row and word position
# per column to a 1D array of words. This will also remove skips
# and OoV words
doc_ids = np.arange(pruned.shape[0])
flattened, (doc_ids,) = corpus.compact_to_flat(pruned, doc_ids)

Pre-trained Word Vectors\
**TODO**: add pre-trained words when given

In [75]:
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
#!pip install gensim
assert flattened.min() >= 0
# Fill in the pretrained word vectors
n_dim = 30

import numpy as np
pretrained_filename= '/content/lda2vec/covid_embeddings/covid_19_w2v_embeds_30.model'


vectors, s, f = corpus.compact_word_vectors(vocab, pretrained_filename,top=10027)
#model = Word2Vec.load(pretrained_filename)
#print(model.wv.syn0)

print(vectors)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/gensim/utils.py:488: DeprecationWarning: Call to deprecated `cum_table` (Attribute will be removed in 4.0.0, use self.vocabulary.cum_table instead).
  setattr(self, attrib, None)
/content/lda2vec/lda2vec/corpus.py:541: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  n_dim = model.syn0.shape[1]
/content/lda2vec/lda2vec/corpus.py:544: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  data += model.syn0.mean()
/content/lda2vec/lda2vec/corpus.py:546: DeprecationWarning: Call

[[-0.45716345  2.0606704  -0.13628319 ...  0.852569    1.0241576
  -1.298093  ]
 [-0.68852    -1.874537   -0.6358273  ...  0.22736378  1.1254219
   0.60855556]
 [ 0.714706   -0.79441506 -0.17988043 ... -0.7308698  -1.0751224
  -1.828733  ]
 ...
 [-0.641551   -0.49909088 -0.0076562  ...  1.0441295  -0.3340283
  -0.02855558]
 [ 0.06053621  0.07970229  0.00658272 ...  0.12321697  0.12195609
   0.32120553]
 [ 0.22220062  1.3694954  -0.6697449  ... -1.6497272  -1.6762612
   1.9620782 ]]


Save all files (uncomment for vectors if pre-trained included)

In [0]:
# Save all of the preprocessed files
pickle.dump(vocab, open('vocab.pkl', 'wb'))
pickle.dump(corpus, open('corpus.pkl', 'wb'))
np.save("flattened", flattened)
np.save("doc_ids", doc_ids)
np.save("pruned", pruned)
np.save("bow", bow)


In [0]:
np.save("vectors", vectors)

In [0]:
print(tokens.shape)
print(vocab)

(744, 100000)
{0: '2019', 1: 'novel', 2: 'coronavirus', 3: 'infection', 4: 'gastrointestinal', 5: 'tract', 6: 'since', 7: 'end', 8: 'december', 9: 'cluster', 10: 'patients', 11: 'pneumonia', 12: 'unknown', 13: 'origin', 14: 'reported', 15: 'wuhan', 16: 'hubei', 17: 'province', 18: 'china', 19: 'feb', 20: '17th', 21: '2020', 22: 'statistical', 23: 'data', 24: 'show', 25: 'outbreak', 26: 'constitutes', 27: 'epidemic', 28: 'threat', 29: 'exponential', 30: 'increase', 31: 'reached', 32: '75114', 33: 'confirmed', 34: 'cases', 35: '2239', 36: 'deaths', 37: 'different', 38: 'sarscov', 39: 'severe', 40: 'acute', 41: 'respiratory', 42: 'syndrome', 43: 'merscov', 44: 'middle', 45: 'east', 46: 'initial', 47: 'presentations', 48: 'chief', 49: 'complain', 50: 'covid19', 51: 'symptoms', 52: 'call', 53: 'upon', 54: 'firstline', 55: 'medical', 56: 'staff', 57: 'cautious', 58: 'pay', 59: 'attention', 60: 'untypical', 61: 'especially', 62: 'area', 63: 'besides', 64: 'viral', 65: 'nucleic', 66: 'acids', 

In [0]:
%cd /content/lda2vec
#data_dir = os.getenv('data_dir', '../data/')
fn_vocab = 'vocab.pkl'
fn_corpus = 'corpus.pkl'
fn_flatnd = 'flattened.npy'
fn_docids = 'doc_ids.npy'
#fn_vectors = 'vectors.npy'
vocab = pickle.load(open(fn_vocab, 'rb'))
corpus = pickle.load(open(fn_corpus, 'rb'))
flattened = np.load(fn_flatnd)
doc_ids = np.load(fn_docids)
vectors = np.load('/content/lda2vec/vectors.npy')

/content/lda2vec


##Defining Model Parameters



>> Parameters 
1.   Number of documents
2.   Number of unique words
3.   Dirichlet Prior 
4.   Number of topics
5.   Batchsize
6.   power for negative sampling 
7.   flag to include pre-trained or not
8.   sampling temperature
9.   number of dimensions of word vectors
10.  number of tokens in each document
11.  token frequencies



In [0]:
# Model Parameters
# Number of documents
n_docs = doc_ids.max() + 1
# Number of unique words in the vocabulary
n_vocab = flattened.max() + 1
# 'Strength' of the dircihlet prior; 200.0 seems to work well
clambda = 200.0
# Number of topics to fit
n_topics = int(os.getenv('n_topics', 10))
batchsize = 32
# Power for neg sampling
power = float(os.getenv('power', 0.75))
# Intialize with pretrained word vectors
pretrained = bool(int(os.getenv('pretrained', True)))
# Sampling temperature
temperature = float(os.getenv('temperature', 1.0))
# Number of dimensions in a single word vector
n_units = int(os.getenv('n_units', 30))
# Get the string representation for every compact key
words = corpus.word_list(vocab)[:n_vocab]
# How many tokens are in each document
doc_idx, lengths = np.unique(doc_ids, return_counts=True)
doc_lengths = np.zeros(doc_ids.max() + 1, dtype='int32')
doc_lengths[doc_idx] = lengths
# Count all token frequencies
tok_idx, freq = np.unique(flattened, return_counts=True)
term_frequency = np.zeros(n_vocab, dtype='int32')
term_frequency[tok_idx] = freq

In [0]:
for key in sorted(locals().keys()):
    val = locals()[key]
    if len(str(val)) < 100 and '<' not in str(val):
        print(key, val)

# Training the LDA2vec Model

In [0]:
model = LDA2Vec(n_documents=n_docs, n_document_topics=n_topics,
                n_units=n_units, n_vocab=n_vocab, counts=term_frequency,
                n_samples=15, power=power, temperature=temperature)

Add pretrained data

In [78]:
pretrained = True
'''
if os.path.exists('lda2vec.hdf5'):
    print("Reloading from saved")
    serializers.load_hdf5("lda2vec.hdf5", model)
'''    
print(model.sampler.W.data)
if pretrained:      
    model.sampler.W.data[:, :] = vectors[:n_vocab, :]

[[4.8262525e-01 5.1634759e-01 6.1177611e-02 ... 2.0399883e-01
  8.0873263e-01 8.8757867e-01]
 [3.3343765e-01 8.7369537e-01 5.5628747e-01 ... 7.8630936e-01
  5.7039726e-01 4.2673543e-02]
 [9.3939531e-01 1.1837331e-01 6.4066839e-01 ... 9.0106750e-01
  8.8883007e-01 5.3518818e-04]
 ...
 [2.0887834e-01 9.8987228e-01 5.8763146e-01 ... 1.8661645e-01
  6.8697169e-02 1.4386970e-01]
 [6.7988497e-01 7.1481246e-01 3.2514563e-01 ... 7.0059878e-01
  7.9054904e-01 3.3337933e-01]
 [7.1136072e-02 6.2873799e-01 1.7077924e-01 ... 1.6059573e-01
  2.8287187e-01 3.0598396e-02]]


Adam Optimizer

In [79]:
model.to_gpu()
optimizer = O.Adam()
optimizer.setup(model)
clip = chainer.optimizer.GradientClipping(5.0)
optimizer.add_hook(clip)

/usr/local/lib/python3.6/dist-packages/chainer/optimizer.py:983: DeprecationWarning: chainer.optimizer.GradientClipping is deprecated from v4. Use chainer.optimizer_hooks.GradientClipping instead.
  DeprecationWarning)


In [0]:
j = 0
epoch = 0
fraction = batchsize * 1.0 / flattened.shape[0]
progress = shelve.open('progress.shelve')

Training with cuda


*   Measure coherence between topics every 100 iterations
*   this is for only one epoch
*   save the model
*   backpropagation style 



In [81]:
n_epochs = 1
for epoch in range(n_epochs):
    data = prepare_topics(cuda.to_cpu(model.mixture.weights.W.data).copy(),
                          cuda.to_cpu(model.mixture.factors.W.data).copy(),
                          cuda.to_cpu(model.sampler.W.data).copy(),
                          words)
    top_words = print_top_words_per_topic(data)
    if j % 100 == 0 and j > 100:
        coherence = topic_coherence(top_words)
        for j in range(n_topics):
            print(j, coherence[(j, 'cv')])
        kw = dict(top_words=top_words, coherence=coherence, epoch=epoch)
        progress[str(epoch)] = pickle.dumps(kw)
    data['doc_lengths'] = doc_lengths
    data['term_frequency'] = term_frequency
    np.savez('topics.pyldavis', **data)
    print(epoch)
    for d, f in utils.chunks(batchsize, doc_ids, flattened):
        t0 = time.time()
        model.cleargrads()
        #optimizer.use_cleargrads(use=False)
        l = model.fit_partial(d.copy(), f.copy())
        print("after partial fitting:", l)
        prior = model.prior()
        loss = prior * fraction
        loss.backward()
        optimizer.update()
        msg = ("J:{j:05d} E:{epoch:05d} L:{loss:1.3e} "
               "P:{prior:1.3e} R:{rate:1.3e}")
        prior.to_cpu()
        loss.to_cpu()
        t1 = time.time()
        dt = t1 - t0
        rate = batchsize / dt
        logs = dict(loss=float(l), epoch=epoch, j=j,
                    prior=float(prior.data), rate=rate)
        print(msg.format(**logs))
        j += 1
    serializers.save_hdf5("lda2vec.hdf5", model)

Streaming output truncated to the last 5000 lines.
J:00095 E:00000 L:4.072e+04 P:-1.542e+04 R:7.943e+02
after partial fitting: 31044.2
J:00096 E:00000 L:3.104e+04 P:-1.542e+04 R:7.957e+02
after partial fitting: 23956.664
J:00097 E:00000 L:2.396e+04 P:-1.542e+04 R:8.853e+02
after partial fitting: 35874.06
J:00098 E:00000 L:3.587e+04 P:-1.542e+04 R:8.852e+02
after partial fitting: 31116.072
J:00099 E:00000 L:3.112e+04 P:-1.543e+04 R:9.455e+02
after partial fitting: 32813.605
J:00100 E:00000 L:3.281e+04 P:-1.543e+04 R:8.653e+02
after partial fitting: 26902.123
J:00101 E:00000 L:2.690e+04 P:-1.543e+04 R:8.810e+02
after partial fitting: 19111.09
J:00102 E:00000 L:1.911e+04 P:-1.543e+04 R:8.221e+02
after partial fitting: 36480.293
J:00103 E:00000 L:3.648e+04 P:-1.543e+04 R:8.977e+02
after partial fitting: 34672.96
J:00104 E:00000 L:3.467e+04 P:-1.543e+04 R:9.437e+02
after partial fitting: 22831.965
J:00105 E:00000 L:2.283e+04 P:-1.543e+04 R:9.474e+02
after partial fitting: 29632.785
J:00106 

In [0]:
#!zip -r myzip.zip /content/lda2vec
#from google.colab import files
#files.download('myzip.zip')

In [82]:
npz = np.load(open('/content/lda2vec/topics.pyldavis.npz', 'rb'))
dat = {k: v for (k, v) in npz.items()}
dat['vocab'] = dat['vocab'].tolist()
print(dat['vocab'])

['out_of_vocabulary', 'out_of_vocabulary', '<SKIP>', 'covid19', 'patients', 'coronavirus', 'disease', 'cases', '2019', 'china', 'sarscov2', 'clinical', 'health', 'respiratory', 'infection', 'outbreak', 'severe', 'novel', '2020', 'wuhan', 'epidemic', 'treatment', 'pneumonia', 'virus', 'acute', 'results', 'transmission', 'syndrome', 'control', 'confirmed', 'data', 'february', 'spread', 'risk', 'days', 'study', 'public', 'case', 'symptoms', 'methods', 'reported', 'care', 'infected', 'viral', 'number', 'management', 'may', 'pandemic', 'first', 'measures', 'group', 'countries', 'also', 'prevention', 'early', 'january', 'characteristics', 'caused', 'diagnosis', 'patient', 'findings', 'medical', 'emergency', 'new', '2019ncov', 'world', 'hubei', 'among', 'including', 'time', 'enditag', 'itag', 'hospital', 'december', 'chest', 'two', 'based', 'analysis', 'high', 'higher', 'features', 'using', 'current', 'positive', 'lung', 'fever', 'province', 'potential', 'epidemiological', 'global', 'period',

Print top 10 words for each topic

In [83]:
top_n = 100
topic_to_topwords = {}
for j, topic_to_word in enumerate(dat['topic_term_dists']):
    top = np.argsort(topic_to_word)[::-1][:top_n]
    msg = 'Topic %i '  % j
    top_words = [dat['vocab'][i].strip()[:35] for i in top]
    msg += ' '.join(top_words)
    print(msg)
    topic_to_topwords[j] = top_words

Topic 0 nervous reproductive interferon respiratory inflammatory infectious immune prevention neonatal fetal interleukin gastrointestinal severity transmissible diarrhea kidney cytopathic communicable cytokine contagious immunodeficiency type intestinal acute diarrhoea die organ morbidity neurological diabetes digestive plays mortality animal disease cause chronic serum cardiovascular herd renal health risk causing play clinical depends urinary pathogenesis systemic lung newborn pathological mellitus control allergic liver day subclinical fulminant fetus epidemic hepatitis fatal causes virulence biosafety encephalopathy alveolar metapneumovirus proinflammatory induces etiological leukocyte spinal manifests burden multiorgan prevalence mouse genital cns lymphocyte indicator nutritional electrolyte correlate lethal coronavirus murine ang susceptibility ocular pulmonary fecal ldh airway avian associated causative
Topic 1 cell cells tract des extracorporeal onto nucleic mucous fluids speci

## Visualize Topics 
>> lower λ to get rare words appearing only on selected topics

In [0]:
#!pip install pyLDAvis
import pyLDAvis
pyLDAvis.enable_notebook()
#warnings.filterwarnings('ignore')
import math

c = 0
for term in dat['term_frequency']:
  print(words[c],term)
  c+=1
#dat['topic_term_dists'] = dat['topic_term_dists'] / dat['topic_term_dists'].sum(axis=1)[:, None]
prepared_data = pyLDAvis.prepare(dat['topic_term_dists'], dat['doc_topic_dists'], 
                                 dat['doc_lengths'] * 1.0, dat['vocab'], dat['term_frequency'] * 1.0, mds='tsne')

In [85]:
pyLDAvis.display(prepared_data)

## Infer topics for new document

Parse new doc

In [0]:
corpus = Corpus()
new_doc_text = ['this is a demo text, but I have to replace this with loaded new document']
def clean(line):
    return ' '.join(w for w in line.split() if not any(t in w for t in bad))

new_doc_text = [str(clean(d)) for d in new_doc_text if len(str(clean(d))) > 0]
new_tokens,_ = preprocess.tokenize(new_doc_text, max_length, merge=False,n_process=4)
corpus.update_word_count(new_tokens)
corpus.finalize()
compact = corpus.to_compact(new_tokens)
# Remove extremely rare words
pruned = corpus.filter_count(compact, min_count=30)
bow = corpus.compact_to_bow(pruned)
clean = corpus.subsample_frequent(pruned)
new_doc_id = np.arange(pruned.shape[0])
flattened, (new_doc_id,) = corpus.compact_to_flat(pruned, new_doc_id)


/content/lda2vec/lda2vec/preprocess.py:77: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for row, doc in progress(enumerate(nlp.pipe(texts, **kwargs))):


In [0]:
n_epochs = 1

for epoch in range(1):
    data = prepare_topics(cuda.to_cpu(model.mixture.weights.W.data).copy(),
                          cuda.to_cpu(model.mixture.factors.W.data).copy(),
                          cuda.to_cpu(model.sampler.W.data).copy(),
                          words)
    top_words = print_top_words_per_topic(data)
    data['doc_lengths'] = doc_lengths
    data['term_frequency'] = term_frequency
    print(epoch)
    for d, f in utils.chunks(batchsize, np.array(new_doc_id), flattened):
        t0 = time.time()
        model.cleargrads()
        #optimizer.use_cleargrads(use=False)
        l = model.fit_partial(d.copy(), f.copy(),update_only_docs=True)
        print(l)
        print("after partial fitting:", l)
        prior = model.prior()
        loss = prior * fraction
        loss.backward()
        optimizer.update()
        msg = ("J:{j:05d} E:{epoch:05d} L:{loss:1.3e} "
               "P:{prior:1.3e} R:{rate:1.3e}")
        prior.to_cpu()
        loss.to_cpu()
        t1 = time.time()
        dt = t1 - t0
        rate = batchsize / dt
        logs = dict(loss=float(l), epoch=epoch, j=j,
                    prior=float(prior.data), rate=rate)
        print(msg.format(**logs))
        j += 1

In [0]:
for elem in corpus:
  print(lda_model[corpus[elem]])